In [14]:
import mne
import mne.io.eeglab

montage = mne.channels.read_montage(kind='standard_1005')
s_freq = 5000
chanlabels = ['Fp1',	'Fp2',	'F3',	'F4',	'C3',	'C4',	'P3',	'P4',	
              'O1',	'O2',	'F7',	'F8',	'T7',	'T8',	'P7',	'P8',	
              'Fz',	'Cz',	'Pz',	'Oz',	'FC1',	'FC2',	'CP1',	'CP2',	
              'FC5',	'FC6',	'CP5',	'CP6',	'TP9',	'TP10', 'POz', 'ECG']
ch_types = ['eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	
            'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	
            'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	
            'eeg',	'eeg',	'eeg',	'eeg',	'eeg',	'eeg', 'eeg', 'ecg']
montage = mne.channels.read_montage(kind='standard_1005')
info_custom = mne.create_info(chanlabels, s_freq, ch_types, montage=montage)
info_custom['description'] = 'Simultaneously recorded data with customised info file'

data_path1 = '/Volumes/INTENSO/DPX_EEG_fMRI/EEG/All_subjects2/Sub1_B.set' 
data_path2 = '/Volumes/INTENSO/DPX_EEG_fMRI/EEG/All_subjects2/Sub2_B.set' 

In [15]:
epochs1 = mne.io.read_epochs_eeglab(data_path1, events=None, event_id=None, montage=montage, eog=(), verbose=None, 
                                   uint16_codec=None)
conv_epochs1 = mne.EpochsArray(epochs1, info=info_custom, tmin=-0.25)
evoked1 = conv_epochs1.average()

epochs2 = mne.io.read_epochs_eeglab(data_path2, events=None, event_id=None, montage=montage, eog=(), verbose=None, 
                                   uint16_codec=None)
conv_epochs2 = mne.EpochsArray(epochs2, info=info_custom, tmin=-0.25)
evoked2 = conv_epochs2.average()

list = [evoked1, evoked2]
evoked = mne.grand_average(list)

Extracting parameters from /Volumes/INTENSO/DPX_EEG_fMRI/EEG/All_subjects2/Sub1_B.set...
The following EEG sensors did not have a position specified in the selected montage: [u'ECG']. Their position has been left untouched.
44 matching events found
0 projection items activated
Ready.
44 matching events found
0 projection items activated
0 bad epochs dropped


<ipython-input-15-42ef1f6f0791>:2: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: [u'ECG']. Their position has been left untouched.
  uint16_codec=None)


Extracting parameters from /Volumes/INTENSO/DPX_EEG_fMRI/EEG/All_subjects2/Sub2_B.set...
The following EEG sensors did not have a position specified in the selected montage: [u'ECG']. Their position has been left untouched.
43 matching events found
0 projection items activated
Ready.
43 matching events found
0 projection items activated
0 bad epochs dropped
Identiying common channels ...
all channels are corresponding, nothing to do.


<ipython-input-15-42ef1f6f0791>:7: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: [u'ECG']. Their position has been left untouched.
  uint16_codec=None)


In [17]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os.path as op
from scipy.spatial import Delaunay
import ipyvolume as p3

evoked = evoked.pick_types(meg=False, eeg=True)

In [18]:
# transform evoked into colors
cmap = plt.get_cmap('RdBu_r')
data = np.array(evoked.data)
n_chans, n_times = data.shape
vmax = data.max()
data[data<=-vmax] = -vmax
data[data>=vmax] = vmax
data -= -vmax
data /= 2*vmax
colors = np.reshape(cmap(data.ravel())[:, :3], [n_chans, n_times, 3])
colors = colors.transpose(1, 0, 2)

# set up 3D mesh
pos3D = np.array([ch['loc'][:3] for ch in evoked.info['chs']])
tri3D = Delaunay(pos3D).convex_hull

In [19]:
# plot
fig = p3.figure()
s = p3.plot_trisurf(pos3D[:, 0], pos3D[:, 1], [pos3D[:, 2] for _ in range(n_times)],
                    triangles=tri3D, color=colors)
p3.xyzlim(pos3D.min(), pos3D.max())
p3.animation_control(s, interval=10)
p3.show()

A Jupyter Widget

In [20]:
# set up 2D topography
pos = mne.find_layout(evoked.info).pos[:, :3]
pos -= pos.mean(0, keepdims=True)
pos[:, 2] = (1-(pos[:, 0] ** 2 + pos[:, 1] ** 2))/1e3
tri2D = Delaunay(pos).convex_hull

In [21]:
# plot
fig = p3.figure()
s = p3.plot_trisurf(pos[:, 0], pos[:, 1], [pos[:, 2] for _ in range(n_times)],
                    triangles=tri2D, color=colors)
p3.xyzlim(pos.min(), pos.max())
p3.animation_control(s, interval=10, add=True)
p3.show()

A Jupyter Widget